<a href="https://www.kaggle.com/code/abdelrahmanm2003/fasttext-embeddings?scriptVersionId=174742745" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# !pip install -U gensim
# !pip install nltk
# !pip install spacy
# !install --upgrade scipy
# !pip install gensim[all]
# !python -m spacy download 'en_core_web_sm'
# !pip install -U scipy
# !pip install --upgrade pip

from gensim.test.utils import datapath
from gensim.models import FastText
import pandas as pd
import nltk
from gensim.parsing.preprocessing import STOPWORDS
import re
import spacy
import numpy as np
import tqdm 
import pandas as pd
import pandas as pd

In [ ]:
# # # !pip uninstall scipy -y
# # !pip install scipy
# # # !pip install --upgrade pip
# !pip install scipy==1.10.0

# !pip install gensim 

In [ ]:
STOPWORDS

In [ ]:
data_tip=pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json', lines=True,)[:3000]


In [ ]:
len(data_tip)

In [ ]:
tip_text = data_tip['text']

In [ ]:
tip_text[:3]

In [ ]:
import spacy

def process_text(document,stopwords=STOPWORDS):
    nlp = spacy.load("en_core_web_sm")
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document)) 
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    document = document.lower() 
    doc_spacy=nlp(document)
    lemma_txt = ["".join(token.lemma_) for token in tqdm.tqdm(doc_spacy)]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in STOPWORDS]
    lemma_no_stop_txt = [word for word in lemma_no_stop_txt if len(word) > 3]
    clean_txt = ' '.join(lemma_no_stop_txt)
    return lemma_no_stop_txt

tip_text_preprocessed = tip_text.apply(process_text)



In [ ]:
tip_text_preprocessed

In [ ]:
model3= FastText(vector_size=300,workers=20,window=7,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=100)

In [ ]:
model3.save('/kaggle/working/custom_trained_model.h5')

In [ ]:
model3.ns_exponent #PROBABILITY OF NEGATIVE WORD SAMPLING

In [ ]:
test_words="Pizza Burger Coffee Restaurant Delivery Review Menu Rating Service Ambience".split()
test_words=list(np.array(test_words).astype(str))

In [ ]:
def test_words_andmodel_updating(model, test_words):
    similar_words = []
    un_similar_words=[]
    embedding_all_test_words_list_separable=[]
    for word in test_words:
        similar_word = model.wv.most_similar(positive=[word], topn=10)
        unsimilar_word =model.wv.most_similar(negative=[word], topn=10)
        embedding_each_test_word=model.wv[word]
        embedding_all_test_words_list_separable.append(embedding_each_test_word)
        similar_words.append(similar_word)
        un_similar_words.append(unsimilar_word)
        model3= FastText(vector_size=312,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)
    return pd.DataFrame({"test_embedding":embedding_all_test_words_list_separable,"most_similar":similar_words , "most_opposite":un_similar_words})


In [ ]:
test_custom_trained_model_updating=test_words_andmodel_updating(model3,test_words)

In [ ]:
test_custom_trained_model_updating

In [ ]:

pd.set_option('display.max_colwidth', None)
test_custom_trained_model_updating.to_csv('/kaggle/working/my_trained_model_test__.csv', index=False)


In [ ]:
test_custom_trained_model_updating

In [ ]:
np.array(test_custom_trained_model_updating['test_embedding'][0]).shape

# Facebook pretrained model

In [ ]:
!pip install fasttext[all]

In [ ]:
import fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore') 
model_path='/kaggle/working/cc.en.300.bin.gz'


In [ ]:
import gensim
import gzip
with gzip.open('/kaggle/working/cc.en.300.bin.gz', 'rb') as f:
      f.read()
 


In [ ]:
from gensim.models import fasttext

model_pre_trained = fasttext.load_facebook_model('/kaggle/working/cc.en.300.bin')

In [ ]:
model_pre_trained.wv['hello']

In [ ]:
def test_words_pretrain_model(model, test_words):
    similar_words = []
    un_similar_words=[]
    embedding_all_test_words_list_separable=[]
    for word in test_words:
        similar_word = model.wv.most_similar(positive=[word], topn=10)
        unsimilar_word =model.wv.most_similar(negative=[word], topn=10)
        embedding_each_test_word=model.wv[word]
        embedding_all_test_words_list_separable.append(embedding_each_test_word)
        similar_words.append(similar_word)
        un_similar_words.append(unsimilar_word)
#         model3= FastText(vector_size=312,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)
    return pd.DataFrame({"test_word":test_words,"test_embedding":embedding_all_test_words_list_separable,"most_similar":similar_words , "most_opposite":un_similar_words})


In [ ]:
pd.set_option('display.max_colwidth', None)
pretrain_model_testing=test_words_pretrain_model(model_pre_trained,test_words)
pretrain_model_testing

In [ ]:
pretrain_model_testing.to_csv('pretrain_model_testing')